# Set 1 - The Cryptopals Crypto Challenges

Cryptopals Rule:
>Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing.

## Encodings

- ASCII (256). [Table](https://www.ascii-code.com/compact)
- HEX (16)

## Numbering Systems

1. Binary (base 2)
2. [Octal](https://www.electronics-tutorials.ws/binary/bin_4.html) (base 8)
3. Decimal (base 10)
4. Hexadecimal (base 16)

| Binary | 0 | 1 | 10 | 11 | 100 | 101 | 110 | 111 | 1000 | 1001 | 1010 | 1011 | 1100 | 1101 | 1110 | 1111 | 10000 |
| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:|
| Decimal | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9  | 10 | 11 | 12 | 13 | 14 | 15 | 16 |
| Hexadecimal | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9  | A | B | C | D | E | F | 10 |

### Example

Represent the decimal number 512 in hex: `512 = 2 x 16^2 + 0 x 16^1 + 0 x 160 = 200`

## Challenge 1 - Convert hex to base64

In [2]:
import base64

def convert_hex_to_base64(str):
  # first decode string from hex representation
  raw_bytes = bytearray.fromhex(str)
  # now encode bytes to base64 encoding
  b64_bytes = base64.b64encode(raw_bytes)

  return b64_bytes

In [3]:
# https://docs.python.org/3/library/stdtypes.html#bytes.decode
assert convert_hex_to_base64("49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d").decode() == "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"
print(f'Set 1 Challenge 1 passing')

Set 1 Challenge 1 passing


## Challenge 2 - Fixed XOR

>In cryptography, the simple XOR cipher is a type of additive cipher, an encryption algorithm that operates according to the principles:

```
A ^ 0 = A
A ^ A = 0
A ^ B = B ^ A
(A ^ B) ^ C = A ^ (B ^ C)
(B ^ A) ^ A = B ^ 0 = B
```

>where `^` denotes the exclusive disjunction (XOR) operation. This operation is sometimes called modulus 2 addition (or subtraction, which is identical). With this logic, a string of text can be encrypted by applying the bitwise XOR operator to every character using a given key. To decrypt the output, merely reapplying the XOR function with the key will remove the cipher.

*[Source](https://en.wikipedia.org/wiki/XOR_cipher)*

### XOR Cipher Trace Table

| Plaintext | Key | Ciphertext |
| - | - | - |
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

In [4]:
def xor(bytes1, bytes2):
  return bytes([b1 ^ b2 for b1, b2 in zip(bytes1, bytes2)])

In [5]:
assert xor(bytearray.fromhex("1c0111001f010100061a024b53535009181c"), bytearray.fromhex("686974207468652062756c6c277320657965")).hex() == "746865206b696420646f6e277420706c6179"
print(f'Set 1 Challenge 2 passing')

Set 1 Challenge 2 passing


## Challenge 3 - Single-byte XOR cipher

In [6]:
from string import ascii_lowercase
from collections.abc import Iterable

letter_frequency = { 'e': 12.70, 't': 9.05, 'a': 8.16, 'o': 7.50, 'i': 6.96, 'n': 6.74, 's': 6.32, 'h': 6.09, 'r': 5.98, 'd': 4.25, 'l': 4.02, 'c': 2.78, 'u': 2.75, 'm': 2.40, 'w': 2.36, 'f': 2.22, 'g': 2.01, 'y': 1.97, 'p': 1.92, 'b': 1.49, 'v': 0.97, 'k': 0.77, 'j': 0.15, 'x': 0.15, 'q': 0.09, 'z': 0.07 }
allowed_characters = ['.', ',', '\'', '"', ' ', '!', '?', '-']

def charater_frequency_score(str):
  score = 0
  for c in str:
    if c < 0 or c > 127:
      return -1000
    c = chr(c)
    if not c.isalnum() and c not in allowed_characters:
        score -= 50
    elif c.isalpha():
        score += letter_frequency[c.lower()] * 10

  return score

def single_byte_xor_cipher(cipher_str):
  if isinstance(cipher_str, str): cipher_str = bytearray.fromhex(cipher_str)
  result = None

  for n in range(0, 256):
    # When using a single character/int for key, just duplicate to the length of the cipher
    key = n.to_bytes(1, byteorder='big')
    keystream = key * len(cipher_str)
    xored_bytes = xor(cipher_str, keystream)
    score = charater_frequency_score(xored_bytes)
    if result == None or result["score"] < score: 
      result = {"plaintext": xored_bytes, "key": n, "score": score}

  return result

# cipher = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"
# print(single_byte_xor_cipher(cipher))
# cipher = bytearray.fromhex(cipher) # hex decode
# keystream = (88).to_bytes(1, "big") * len(cipher)
# xor(cipher, keystream)

In [7]:
assert single_byte_xor_cipher("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736")["key"] == 88
print(f'Set 1 Challenge 3 passing')

Set 1 Challenge 3 passing


## Challenge 4 - Detect single-character XOR

In [8]:
def detect_single_character_xor():
  file = open('4.txt', 'r')
  lines = file.readlines()
  highest_score = None
  for line in lines:
      result = single_byte_xor_cipher(line)
      if highest_score == None or highest_score["score"] < result["score"]:
        highest_score = result

  return highest_score

detect_single_character_xor()

{'plaintext': b'Now that the party is jumping\n', 'key': 53, 'score': 1350.8}

In [9]:
assert detect_single_character_xor()["key"] == 53
print(f'Set 1 Challenge 4 passing')

Set 1 Challenge 4 passing


## Challenge 5 - Implement repeating-key XOR

In [10]:
import binascii

def repeating_key_xor(s, key):
  if isinstance(s, str): s = bytes(s, "utf-8")
  if isinstance(key, str): key = bytes(key, "utf-8")
  cipher = []

  for i in range(len(s)):
    b = s[i]
    # Rotate through the key's characters
    c = key[i % len(key)]
    cipher.append(b ^ c)

  pp = binascii.hexlify(bytes(cipher)).decode('ascii')
  # print(pp)
  return pp

# cipher_array = repeating_key_xor("Burning 'em, if you ain't quick and nimble", "ICE")
# binascii.hexlify(cipher_array).decode('ascii')

In [11]:
input5 = """Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal"""
output5 = """0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"""

assert repeating_key_xor(input5, "ICE") == output5
print(f'Set 1 Challenge 5 passing')

Set 1 Challenge 5 passing


## Challenge 6 - Break repeating-key XOR

In [12]:
def hamming_distance(bytes1, bytes2):
  distance = 0
  for b1, b2 in zip(bytes1, bytes2):
    diff = bin(b1 ^ b2)
    count = diff.count('1') # a 1 denoting a difference, see above table of XOR
    distance += count

  return distance

assert hamming_distance(b'''this is a test''', b'''wokka wokka!!!''') == 37

In [13]:
def to_blocks(lst, n):
  for i in range(0, len(lst), n):
      yield lst[i:i + n]

# List comprehension
# def to_blocks(lst, n):
  # return [lst[i:i + n] for i in range(0, len(lst), n)]

assert list(to_blocks([1,2,3,4,5,6,7,8], 2)) == [[1,2],[3,4],[5,6],[7,8]]
assert list(to_blocks([1,2,3,4,5,6,7,8], 3)) == [[1,2,3],[4,5,6],[7,8]]
assert list(to_blocks([], 4)) == []

In [31]:
def find_keysize(ciphertext, min_length = 2, max_length = 40):
  dists_keys = []
  for KEYSIZE in range(min_length, max_length):
    slice_size = 2 * KEYSIZE
    # // floor division 100 // 3 = 33
    measurements = len(ciphertext) // slice_size - 1
    score = 0
    for i in range(measurements):
      first = slice(i * slice_size, i * slice_size + KEYSIZE)
      second = slice(i * slice_size + KEYSIZE, i * slice_size + 2 * KEYSIZE)

      score += hamming_distance(ciphertext[first], ciphertext[second])
    score /= KEYSIZE
    score /= measurements
    dists_keys.append({ "score": score, "keysize": KEYSIZE})

  lowest_hamming = sorted(dists_keys, key = lambda d: d["score"])

  return lowest_hamming

def break_repeating_key_xor(ciphertext):
  KEYSIZE = find_keysize(ciphertext)[0]["keysize"]
  # Try the best three keys
  # for KEYSIZE in KEYSIZES[:3]:
  key = bytes()
  plaintext = []
  for i in range(KEYSIZE):
    chunk = single_byte_xor_cipher(bytes(ciphertext[i::KEYSIZE]))
    k = bytes(chunk["key"])
    plaintext.append(chunk["plaintext"])
    key += k

  message = bytes()
  for i in range(max(map(len, plaintext))):
    message += bytes([chunk[i] for chunk in plaintext if len(chunk) >= i + 1])

  return message

In [36]:
file6 = open('6.txt', 'r')
ciphertext6 = base64.b64decode(file6.read())

assert find_keysize(ciphertext6)[0]["keysize"] == 29
assert break_repeating_key_xor(ciphertext6).decode("utf-8").startswith("I'm back and I'm ringin'")

print("Set 1 Challenge 6 passing")
# print(break_repeating_key_xor(ciphertext6))

Set 1 Challenge 6 passing


## Challenge 7 - AES in ECB mode

AES is a block cipher. One key feature of block ciphers is they can only encrypt a specific sized block of plaintext. To encrypt plaintext that is not exactly 128 bits, it needs to use *padding* and a *mode of operation*.

In [23]:
# File is Base64-encoded and encrypted with AES-128 ECB mode
file7 = open('7.txt', 'r')
ciphertext7 = base64.b64decode(file7.read())
key = b'YELLOW SUBMARINE' # bytes('YELLOW SUBMARINE', 'utf-8')

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
cipher = Cipher(algorithms.AES128(key), modes.ECB())
decryptor = cipher.decryptor()
plaintext7 = decryptor.update(ciphertext7) + decryptor.finalize()

In [24]:
assert plaintext7.decode("utf-8").startswith("I'm back and I'm ringin'")
print("Set 1 Challenge 7 passing")
# print(break_repeating_key_xor(ciphertext6))

Set 1 Challenge 7 passing


## Challenge 8 - Detect AES in ECB mode

>In this file are a bunch of hex-encoded ciphertexts.
>
>One of them has been encrypted with ECB.
>
>Detect it.
>
>Remember that the problem with ECB is that it is stateless and deterministic; the same 16 byte plaintext block will always produce the same 16 byte ciphertext.

Electronic codebook (ECB) is the mode of operation that divides the message into blocks of 16 bytes and pads the last block if it's not 16 bytes long.
Encryption is deterministic, encrypting the same block twice leads to the same ciphertext. Also, this means encrypting block by block the resulting ciphertext might have repeating patterns.

In [75]:
file8 = open('8.txt', 'r')

def detect_aes_in_ecb_mode(file):
  lines = file8.readlines()
  duplicates = []
  for line in lines:
    cipher = bytearray.fromhex(line)
    # split each line into 16 byte long chunks and count duplicates?
    blocks = list(to_blocks(cipher, 16))
    hexed = [(binascii.hexlify(h)) for h in blocks]
    duplicates.append({ "sum": sum([ hexed.count(i) for i in hexed ]), "line": cipher })

  max_item = max(duplicates, key=lambda x:x["sum"])
  return max_item

# detect_aes_in_ecb_mode(file8)


In [76]:
assert detect_aes_in_ecb_mode(file8)["sum"] == 22
print("Set 1 Challenge 8 passing")

Set 1 Challenge 8 passing
